## MLeap Scikit-Learn Demo

The goal of this demo is to:
    1. Put together an ML pipeline using scikit transformers, pipeline and feature unions
    2. Train a linear regression to predict listing prices
    3. Demonstrate how to serialize scikit-learn transformers and models to bundle.ml
    4. TODO: use .deploy() to deploy a model to combust cloud
    5. TODO: deserialize the pipeline in Spark
    
Note: MLeap <> Scikit-Learn itegration is experimental. We are planning to release a stable version with mleap-0.7.0

## Background on the Dataset

The dataset used for the demo was pulled together from individual cities' data found [here](http://insideairbnb.com/get-the-data.html). We've also gone ahead and pulled the individual datasets and relevant features into this [research dataset](https://s3-us-west-2.amazonaws.com/mleap-demo/datasources/airbnb.csv) stored as csv.

### Step 0: Load libraries and data

In [ ]:
import uuid

import sys
import pandas as pd

import mleap.sklearn.pipeline
import mleap.sklearn.feature_union
import mleap.sklearn.base
import mleap.sklearn.logistic
import mleap.sklearn.preprocessing.data
from mleap.sklearn.ensemble import forest

from mleap.sklearn.preprocessing.data import FeatureExtractor, NDArrayToDataFrame, ToDense, ReshapeArrayToN1, LabelEncoder

from sklearn.linear_model import LinearRegression
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

import numpy as np

In [ ]:
df = pd.read_csv('/Users/mikhail/combust/mleap-demo/data/airbnb/airbnb.csv', error_bad_lines=False, warn_bad_lines=False)



continuous_features = ["bathrooms",
  "bedrooms",
  "security_deposit",
  "cleaning_fee",
  "extra_people",
  "number_of_reviews",
  "square_feet",
  "review_scores_rating"]

df[continuous_features].describe()

In [ ]:
def _transform_state(state):
    if state in ['NY', 'CA', 'London', 'Berlin', 'TX', 'IL', 'OR', 'DC', 'WA']:
        return state
    return 'Other'

continuous_features = ["bathrooms",
  "bedrooms",
  "security_deposit",
  "cleaning_fee",
  "extra_people",
  "number_of_reviews",
  "square_feet",
  "review_scores_rating"]

categorical_features = ["room_type",
  "host_is_superhost",
  "cancellation_policy",
  "state",
  "instant_bookable"]

df['state'] = df.state.apply(_transform_state)

imputed_continuous_features = ['imp_{}'.format(x) for x in continuous_features]

feature_extractor2_tf = FeatureExtractor(input_scalars=continuous_features, 
                                         output_vector='imputed_features', 
                                         output_vector_items=imputed_continuous_features
                                        )

impute_security_deposit_tf = Imputer(strategy='mean', axis=0)
impute_security_deposit_tf.mlinit(prior_tf=feature_extractor2_tf)

impute_pipeline = Pipeline([
        (feature_extractor2_tf.name, feature_extractor2_tf),
        (impute_security_deposit_tf.name, impute_security_deposit_tf)
    ])
impute_pipeline.mlinit()

# Consider doing this via a feature union
df2 = df.join(pd.DataFrame(impute_pipeline.fit_transform(df), columns=feature_extractor2_tf.output_vector_items))

all_features = imputed_continuous_features + categorical_features

In [ ]:
continuous_features = ["bathrooms",
  "bedrooms",
  "security_deposit",
  "cleaning_fee",
  "extra_people",
  "number_of_reviews",
  "square_feet",
  "review_scores_rating"]
imputed_continuous_features = ['imp_{}'.format(x) for x in continuous_features]

feature_extractor2_tf = FeatureExtractor(input_scalars=continuous_features, 
                                         output_vector='imputed_features', 
                                         output_vector_items=imputed_continuous_features)

impute_security_deposit_tf = Imputer(strategy='mean', axis=0)
impute_security_deposit_tf.mlinit(prior_tf=feature_extractor2_tf)

In [ ]:
df2[['state', 'imp_bathrooms', 'imp_square_feet', 'imp_bedrooms', 'imp_security_deposit', 'imp_cleaning_fee', 'imp_extra_people', 'imp_number_of_reviews', 'imp_review_scores_rating', 'room_type', 'host_is_superhost', 'cancellation_policy', 'instant_bookable', 'price']].describe()


### Step 1: Standardize the data for out demo

In [ ]:
def _transform_state(state):
    if state in ['NY', 'CA', 'London', 'Berlin', 'TX', 'IL', 'OR', 'DC', 'WA']:
        return state
    return 'Other'

def _set_min_of_one(x):
    if x <= 1.0:
        return 1.0
    return x

for x in ['imp_bathrooms', 'imp_square_feet', 'imp_bedrooms', 'imp_security_deposit', 'imp_cleaning_fee', 'imp_extra_people', 'imp_number_of_reviews', 'imp_review_scores_rating']:
    df2[x] = df2[x].apply(_set_min_of_one)

df2[(df2.price >= 50)&(df2.price <= 500)&(df2.imp_bathrooms>0)][['state', 'imp_bathrooms', 'imp_square_feet', 'imp_bedrooms', 'imp_security_deposit', 'imp_cleaning_fee', 'imp_extra_people', 'imp_number_of_reviews', 'imp_review_scores_rating', 'room_type', 'host_is_superhost', 'cancellation_policy', 'instant_bookable', 'price']][:250000].to_csv('/Users/mikhail/Documents/airbnb_for_stream_train.csv', index=False)

df2[(df2.price >= 50)&(df2.price <= 500)&(df2.imp_bathrooms>0)][['state', 'imp_bathrooms', 'imp_square_feet', 'imp_bedrooms', 'imp_security_deposit', 'imp_cleaning_fee', 'imp_extra_people', 'imp_number_of_reviews', 'imp_review_scores_rating', 'room_type', 'host_is_superhost', 'cancellation_policy', 'instant_bookable', 'price']][150000:].to_csv('/Users/mikhail/Documents/airbnb_for_stream_train_update.csv', index=False)


### Step 1.1: Take a look at some summary statistics of the data

In [ ]:
df[['state', 'price']].groupby('state').agg([np.size, np.mean]).sort_values(by=('price', 'size'), ascending=False)[:10]

In [ ]:
price_stats=df[['state', 'price']].groupby('state').agg([np.size, np.mean, np.max]).sort_values(by=('price', 'mean'), ascending=False)
price_stats[price_stats[('price','size')]>25][:10]

In [ ]:
# convert to categorical feature
df['host_is_superhost'] = df['host_is_superhost'].apply(str)
df['instant_bookable'] = df['instant_bookable'].apply(str)

# normalize state
df['state'] = df.state.apply(_transform_state)


### Step 2: Define continous and categorical features and filter nulls

In [ ]:
continuous_features = ["bathrooms",
  "bedrooms",
  "security_deposit",
  "cleaning_fee",
  "extra_people",
  "number_of_reviews",
  "square_feet",
  "review_scores_rating"]

categorical_features = ["room_type",
  "host_is_superhost",
  "cancellation_policy",
  "state",
  "instant_bookable"]


In [ ]:
imputed_continuous_features = ['imp_{}'.format(x) for x in continuous_features]

feature_extractor2_tf = FeatureExtractor(input_scalars=continuous_features, 
                                         output_vector='imputed_features', 
                                         output_vector_items=imputed_continuous_features)

impute_security_deposit_tf = Imputer(strategy='mean', axis=0)
impute_security_deposit_tf.mlinit(prior_tf=feature_extractor2_tf)

impute_pipeline = Pipeline([
        (feature_extractor2_tf.name, feature_extractor2_tf),
        (impute_security_deposit_tf.name, impute_security_deposit_tf)
    ])
impute_pipeline.mlinit()

# Consider doing this via a feature union
df2 = df.join(pd.DataFrame(impute_pipeline.fit_transform(df), columns=feature_extractor2_tf.output_vector_items))

all_features = imputed_continuous_features + categorical_features

### Step 3: Split data into training and validation 

In [ ]:
# First filter out outlier prices
df2 = df2[(df2.price>=50)&(df2.price<=500)]

# Split into training and test
X_train, X_test, y_train, y_test = train_test_split(df2[all_features], df2[['price']], test_size=0.33, random_state=42)

### Step 4: Continous Feature Pipeline

In [ ]:
feature_extractor_tf = FeatureExtractor(input_scalars=imputed_continuous_features, 
                                        output_vector='unscaled_cont_features', 
                                        output_vector_items=["scaled_{}".format(x) for x in imputed_continuous_features])

standard_scaler_tf = StandardScaler()
standard_scaler_tf.mlinit(prior_tf=feature_extractor_tf)

standard_scaler_pipeline = Pipeline([(feature_extractor_tf.name, feature_extractor_tf),
                            (standard_scaler_tf.name, standard_scaler_tf)])
standard_scaler_pipeline.mlinit()

### Step 5: Categorical Feature Pipeline

In [ ]:
# TODO: Need to fix scikit's One-Hot-Encoder to drop the last column of a matrix if we're using it for ML
def _create_le_one_hot_pipeline(feature_name):
    feature_extractor3_tf = FeatureExtractor(input_scalars=[feature_name], 
                                             output_vector='{}_label'.format(feature_name), 
                                             output_vector_items=[feature_name])
                                             

    # Label Encoder for x1 Label 
    label_encoder_tf = LabelEncoder(input_features = feature_extractor3_tf.output_vector_items, output_features='{}_label_le'.format(feature_name))

    # Reshape the output of the LabelEncoder to N-by-1 array
    reshape_le_tf = ReshapeArrayToN1()

    # Vector Assembler for x1 One Hot Encoder
    one_hot_encoder_tf = OneHotEncoder(sparse=False)
    one_hot_encoder_tf.mlinit(prior_tf=label_encoder_tf)

    one_hot_encoder_pipeline_x0 = Pipeline([
                                             (feature_extractor3_tf.name, feature_extractor3_tf),
                                             (label_encoder_tf.name, label_encoder_tf),
                                             (reshape_le_tf.name, reshape_le_tf),
                                             (one_hot_encoder_tf.name, one_hot_encoder_tf)
                                            ])
    
    one_hot_encoder_pipeline_x0.mlinit()
    
    return one_hot_encoder_pipeline_x0

In [ ]:
oh_pipelines = [_create_le_one_hot_pipeline(x) for x in categorical_features]


In [ ]:
oh_fes = [x.steps[-1][1] for x in oh_pipelines]

### Step 6: Assemble our features and feature pipeline

In [ ]:
feature_union = FeatureUnion([
        (standard_scaler_pipeline.name, standard_scaler_pipeline)
    ] + [(x.name, x) for x in oh_pipelines])
feature_union.mlinit()

feature_union.fit(X_train, X_test)

### Step 7: Define our linear regression model

In [ ]:
# Put all of the categorical features into a list
oh_features_lists = [[y[1].output_features for y in x.steps if y[1].op == 'one_hot_encoder'] for x in oh_pipelines]
oh_features = [item for sublist in oh_features_lists for item in sublist]
oh_features

In [ ]:
# Vector Assembler, for serialization purposes only
feature_extractor_lr_model_tf = FeatureExtractor(input_vectors=[feature_extractor_tf] + oh_fes,
                                                 output_vector='input_features',
                                                 output_vector_items=[standard_scaler_tf.output_features] + oh_features
                                                )
feature_extractor_lr_model_tf.skip_fit_transform = True

# Define our linear regression
lr_model = LinearRegression()
lr_model.mlinit(input_features='input_features', prediction_column='price_prediction')

lr_model_pipeline = Pipeline([
        (feature_extractor_lr_model_tf.name, feature_extractor_lr_model_tf),
        (lr_model.name, lr_model)
    ])
lr_model_pipeline.mlinit()

In [ ]:
model_pipeline = Pipeline([(feature_union.name, feature_union),
                            (lr_model_pipeline.name, lr_model_pipeline)])

model_pipeline.mlinit()

In [ ]:
model_pipeline.fit(X_train, y_train)


In [ ]:
model_pipeline.serialize_to_bundle('/tmp', 'scikit-airbnb.lr', init=True)


### Step 9: Define our Random Forest Regression Model

In [ ]:
# Vector Assembler, for serialization purposes only
feature_extractor_rf_model_tf = FeatureExtractor(input_vectors=[feature_extractor_tf],
                                                 output_vector='input_features', 
                                                 output_vector_items=imputed_continuous_features)
feature_extractor_rf_model_tf.skip_fit_transform = True


rf = RandomForestRegressor(max_depth=4, n_estimators=11)
rf.mlinit(input_features=feature_extractor_rf_model_tf.output_vector, prediction_column='price_prediction', feature_names=imputed_continuous_features)

rf_model_pipeline = Pipeline([
        (feature_extractor_rf_model_tf.name, feature_extractor_rf_model_tf),
        (rf.name, rf)
    ])
rf_model_pipeline.mlinit()


In [ ]:
rf_model_pipeline.fit(X_train[imputed_continuous_features], y_train)

### Step 8: Fit our pipeline and regression


In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score
params = {
    "{}__max_depth".format(rf.name): [5, 10],
    "{}__n_estimators".format(rf.name): [10, 15, 20]
}

rf_grid = GridSearchCV(estimator=rf_model_pipeline, param_grid=params, n_jobs=-1)
rf_grid.fit(X_train[imputed_continuous_features], y_train)

In [ ]:
best_rf = rf_grid.best_params_
best_max_depth = best_rf["{}__max_depth".format(rf.name)]
best_n_estimators = best_rf["{}__n_estimators".format(rf.name)]

rf_optimal = RandomForestRegressor(max_depth=best_max_depth, n_estimators=best_n_estimators)

rf_optimal.mlinit(input_features=feature_extractor_rf_model_tf.output_vector, 
                  prediction_column='price_prediction', 
                  feature_names=imputed_continuous_features)

rf_model_pipeline = Pipeline([
        (feature_extractor_rf_model_tf.name, feature_extractor_rf_model_tf),
        (rf_optimal.name, rf_optimal)
    ])
rf_model_pipeline.mlinit()

In [ ]:
model_pipeline.fit(X_train, y_train)
#rf_model_pipeline.fit(X_train[imputed_continuous_features], y_train)

In [ ]:
model_pipeline.steps[-1][1].steps[0][1].input_shapes

### Step 9: Serialize our pipelines to bundle.ml

In [ ]:
# Serialize the linear regression model
model_pipeline.serialize_to_bundle('/tmp', 'scikit-airbnb.lr', init=True)

# Serialiaze the random forest model
#rf_model_pipeline.serialize_to_bundle('/tmp', 'scikit-airbnb.rf', init=True)

In [ ]:
a = [{
      "name": "state",
      "type": "string"
    }, {
      "name": "imp_bathrooms",
      "type": "double"
    }, {
      "name": "imp_square_feet",
      "type": "double"
    }, {
      "name": "imp_bedrooms",
      "type": "double"
    }, {
      "name": "imp_security_deposit",
      "type": "double"
    }, {
      "name": "imp_cleaning_fee",
      "type": "double"
    }, {
      "name": "imp_extra_people",
      "type": "double"
    }, {
      "name": "imp_number_of_reviews",
      "type": "double"
    }, {
      "name": "imp_review_scores_rating",
      "type": "double"
    }, {
      "name": "room_type",
      "type": "string"
    }, {
      "name": "host_is_superhost",
      "type": "string"
    }, {
      "name": "cancellation_policy",
      "type": "string"
    }, {
      "name": "instant_bookable",
      "type": "string"
    }]

In [ ]:
columns = [x['name'] for x in a]
df_tet = pd.DataFrame([("NY", 2.0, 1250.0, 3.0, 50.0, 30.0, 2.0, 56.0, 90.0, "Entire home/apt", "1.0", "strict", "1.0")], columns=columns)

In [ ]:
model_pipeline.predict(df_tet)